---
## Prophet

#### Introduction à Prophet

Prophet est une bibliothèque de prévision de séries temporelles développée par Facebook. Elle est particulièrement adaptée aux séries avec des tendances non linéaires, des variations saisonnières et des événements exceptionnels. 

#### Pourquoi utiliser Prophet ?
- **Tendance :** Modélise les changements à long terme dans les données.
- **Saisonnalité :** Prend en compte les cycles récurrents (hebdomadaires, annuels, etc.).
- **Facilité d'utilisation :** Simple à configurer, même avec des données complexes ou irrégulières.
- **Robustesse :** Gère efficacement les valeurs manquantes et les anomalies.

Prophet est idéal pour des prévisions rapides et fiables, que ce soit pour des données commerciales, climatiques ou autres applications de séries temporelles.


In [1]:
import pandas as pd
import plotly_express as px
from prophet import Prophet
from useful_functions import *
file_path = "./results.json"

In [2]:
# --Tables--
series_train = pd.read_csv("series_train.csv")
series_test = pd.read_csv("series_test.csv")

In [3]:
# --training--
vars = ['date', 'sales']
train_data = series_train[vars].reset_index(drop=True)  
train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

model_prophet = Prophet()
model_prophet.fit(train_data)

future1 = model_prophet.make_future_dataframe(periods=0)
forecast1 = model_prophet.predict(future1)
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
series_train['Train_Forecast_prophet'] = forecast1['yhat'].values

# --testing--
test_data = series_test[['date']].reset_index(drop=True)  
test_data.rename(columns={'date': 'ds'}, inplace=True) 

forecast_test = model_prophet.predict(test_data)

y_test_pred = forecast_test['yhat']
series_test['Test_Forecast_prophet'] = y_test_pred.values

21:25:43 - cmdstanpy - INFO - Chain [1] start processing
21:25:43 - cmdstanpy - INFO - Chain [1] done processing


```python
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], y_test_pred)
        
save_model_results(
    file_path=file_path,
    model_name="Prophet",
    params= 'without exogeneous variables',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```

In [4]:
fig = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'],title="Predictions du train")
fig.show()

In [5]:
fig = px.line(series_test, x='date', y=['sales', 'Test_Forecast_prophet'],title="Predictions du test")
fig.show()

In [6]:
nrmse, mape, mae,r2 = calculate_metrics(series_test['sales'], y_test_pred)
print(nrmse)
print(mape)
print(mae)
print(r2)

0.10937843920566573
1.4338755238509534e+20
265234.6544332331
-0.45950510525269395


Les pics semblent corrects, mais nous restons légèrement au-dessus.  
Les résultats du NRMSE paraissent satisfaisants.  

### Essayons d'intégrer les variables exogènes dans le model :**`onpromotion`**, **`oil_price`**, **`holiday`**.

In [7]:
# --training--
vars = ['date', 'sales','holiday','onpromotion','oil_price']
train_data = series_train[vars].reset_index(drop=True)  
train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

model_prophet = Prophet()
# __exogs__
model_prophet.add_regressor('holiday')
model_prophet.add_regressor('onpromotion')
model_prophet.add_regressor('oil_price')


model_prophet.fit(train_data)

future2 = model_prophet.make_future_dataframe(periods=0)
# __exogs__
future2['holiday'] = train_data['holiday']
future2['onpromotion'] = train_data['onpromotion']
future2['oil_price'] = train_data['oil_price']

forecast2 = model_prophet.predict(future2)
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
y_train_pred = forecast2['yhat']
series_train['Train_Forecast_prophet'] = y_train_pred.values

# --testing--
test_vars =['date','holiday','oil_price','onpromotion']
test_data = series_test[test_vars].reset_index(drop=True)  
test_data.rename(columns={'date': 'ds'}, inplace=True) 

forecast_test = model_prophet.predict(test_data)

y_test_pred = forecast_test['yhat'] 
series_test['Test_Forecast_prophet'] = y_test_pred.values

21:25:44 - cmdstanpy - INFO - Chain [1] start processing
21:25:45 - cmdstanpy - INFO - Chain [1] done processing


```python
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], y_test_pred)
        
save_model_results(
    file_path=file_path,
    model_name="Prophet_exogs",
    params='with exogenous variables',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```

In [8]:
fig = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'],title="Predictions du train")
fig.show()

In [9]:
fig = px.line(series_test, x='date', y=['sales', 'Test_Forecast_prophet'],title="Predictions du test")
fig.show()

In [10]:
nrmse, mape, mae,r2 = calculate_metrics(series_test['sales'], y_test_pred)
print(nrmse)
print(mape)
print(mae)
print(r2)

0.14151367414457225
1.62075222593947e+20
391980.2296472504
-1.4430873125097166


>L'ajout de variables exogènes ne semble pas améliorer la situation.  
>Prophet semble bien capturer les pics, mais reste toujours légèrement au-dessus.


Sauvegarder les predictions

In [11]:
#series_train.to_csv("series_train",index =False, header=True)
#series_test.to_csv("series_test",index =False, header=True)

--- 

# Online approch

Simulons un flux de données : tout les 7 jours, nous avons une semaine de nouvelels dnnées. Prophet n'est pas specialement concu pour uen approche en ligne, alors reentrainons le model tout les 7 jours. 
Les variables exogenes n'aidant pas, on ne les prendra pas en compte

In [ ]:
# --training--
vars = ['date', 'sales']
train_data = series_train[vars].reset_index(drop=True)  
train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

model_prophet = Prophet()
model_prophet.fit(train_data)

future1 = model_prophet.make_future_dataframe(periods=0)
forecast1 = model_prophet.predict(future1)
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
series_train['Train_Forecast_prophet'] = forecast1['yhat'].values

# --testing--

for new_data in data_stream(series_test, chunk_size=7):


    new_data = new_data[vars].reset_index(drop=True)
    new_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

    # --reentrainement apres nouvelles data--
    train_data = pd.concat([train_data, new_data])
    model_prophet = Prophet()

    model_prophet.fit(train_data)

    future = model_prophet.make_future_dataframe(periods=0)

    forecast = model_prophet.predict(future)

    # MAJ
    y_test_pred = forecast.loc[len(series_train):, 'yhat'].values
    series_test.loc[:len(y_test_pred)-1, 'Test_Forecast_prophet_online'] = y_test_pred


21:25:45 - cmdstanpy - INFO - Chain [1] start processing
21:25:45 - cmdstanpy - INFO - Chain [1] done processing
21:25:45 - cmdstanpy - INFO - Chain [1] start processing
21:25:46 - cmdstanpy - INFO - Chain [1] done processing
21:25:46 - cmdstanpy - INFO - Chain [1] start processing
21:25:46 - cmdstanpy - INFO - Chain [1] done processing
21:25:46 - cmdstanpy - INFO - Chain [1] start processing
21:25:46 - cmdstanpy - INFO - Chain [1] done processing
21:25:47 - cmdstanpy - INFO - Chain [1] start processing
21:25:47 - cmdstanpy - INFO - Chain [1] done processing
21:25:47 - cmdstanpy - INFO - Chain [1] start processing
21:25:47 - cmdstanpy - INFO - Chain [1] done processing
21:25:47 - cmdstanpy - INFO - Chain [1] start processing
21:25:48 - cmdstanpy - INFO - Chain [1] done processing
21:25:48 - cmdstanpy - INFO - Chain [1] start processing
21:25:48 - cmdstanpy - INFO - Chain [1] done processing
21:25:48 - cmdstanpy - INFO - Chain [1] start processing
21:25:48 - cmdstanpy - INFO - Chain [1]

In [21]:
fig_train = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'], title="Prédictions sur le train")
fig_train.show()

fig_test = px.line(series_test, x='date', y=['sales', 'Test_Forecast_prophet_online','Test_Forecast_prophet'], title="Prédictions sur le test")
fig_test.show()

>L'approche en ligne est bien meilleurs visuellement

In [22]:
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], series_test['Test_Forecast_prophet_online'])
print(nrmse)
print(mape)
print(mae)
print(r2)

0.07307817373304117
7.560305712611636e+19
126309.73512829006
0.34849531085948604


```python
save_model_results(
    file_path=file_path,
    model_name="Prophet_online",
    params='online toutles 7 jours',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```